**Nombre alumno:** Gemma del Val

# Proyecto *Pump-it-up*

## ¿Qué bombas funcionan y cuáles no?

### Introducción

Actualmente, la población de Tanzania tiene un acceso muy deficiente al agua potable. Aproximadamente el 47% de sus ciudadanos no tienen acceso a esta. Más de $1.4 millones de dólares en ayuda exterior han sido donados al país en un intento de ayudar a solucionar la crisis del agua dulce. Por otro lado, el gobierno de Tanzania no está poniendo una solución a este problema. Una buena proporción de las bombas de agua o no funcionan o apenas lo hacen y necesitan reparación. Muchas personas deben beber agua sucia, llena de patógenos, o caminar varios kilómetros sólo para llegar hasta la bomba de agua subterránea funcional más cercana.

### Objetivo

Haciendo uso de los datos cargados en la web de Taarifa y el Ministerio de Tanzania del Agua, nuestro objetivo es predecir qué bombas funcionan, cuáles no y cuáles necesitan una reparación. Entender cuáles fallarán es importante por los siguientes motivos: 

- Predecir la funcionalidad de todas las bombas de agua subterránea que se encuentran en el territorio con modelos precisos, podría ayudar a ahorrarle al gobierno de Tanzania mucho tiempo y dinero.
- Estos modelos pueden ayudar a reducir el costo de inspección de cada bomba de agua.
- El gobierno puede usar este estudio para saber exactamente cuál es la situación del bombeo de su agua.


El presente trabajo va a estar dividido en tres partes: siendo la primera el Análisis Exploratorio de los Datos, la segunda el Preprocesado de los mismos y la tercera la Selección del Modelo y Entrega.

# I. Análisis Exploratorio de los Datos

## 1. Descripción de los datos
Comenzamos con una pequeña descripción de los datos con los que vamos a trabajar:
### 1.1 Características

* amount_tsh - Cantidad de agua para bombear
* date_recorded - Fecha de inserción de datos
* funder - Quién fundó el pozo
* gps_height - Altitud del pozo
* installer - Organización que lo instaló
* longitude - Coordenadas GPS
* latitude - Coordenadas GPS
* wpt_name - Nombre de la bomba (si es que lo tiene)
* num_private - Número
* basin - Cuenca geográfica
* subvillage - Localización geográfica
* region - Localización geográfica
* region_code - Localización geográfica (en código)
* district_code - Localización geográfica (en código)
* lga - Localización geográfica
* ward - Localización geográfica
* population - Población junto al pozo
* public_meeting - Verdadero/Falso
* recorded_by - Grupo que introduce estos datos
* scheme_management - Quién gestiona la bomba
* scheme_name - Quién maneja la bomba
* permit - Si la bomba está permitida o no
* construction_year - Año de construcción de la bomba
* extraction_type - Tipo de extracción de la bomba
* extraction_type_group - Tipo de extracción de la bomba
* extraction_type_class - Tipo de extracción de la bomba
* management - Cómo se gestiona la bomba
* management_group - Cómo se gestiona la bomba
* payment - Coste del agua
* payment_type - Coste del agua
* water_quality - Calidad del agua
* quality_group - Calidad del agua
* quantity - Cantidad de agua
* quantity_group - Cantidad de agua
* source - Fuente del agua
* source_type - Fuente del agua
* source_class - Fuente del agua
* waterpoint_type - Tipo de bomba
* waterpoint_type_group - Tipo de bomba

### 1.2 Etiquetas

* **functional** - La bomba funciona y no es necesario repararla
* **functional needs repair** - Funciona, pero necesita reparación
* **non functional** - La bomba de agua no funciona

## 2. Importación de librerías y datos

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
%matplotlib inline

from sklearn.metrics import accuracy_score

pd.set_option('display.max_columns', None)
print("Setup Complete")

In [ ]:
pylab.rcParams["figure.figsize"] = (14,8)

In [ ]:
X_train = pd.read_csv("Training_values.csv")
y_train = pd.read_csv("Training_labels.csv")
X_test = pd.read_csv("Test_values.csv")

train_df = X_train.merge(y_train, how='outer', left_index=True, right_index=True)

## 3. Estadística Descriptiva

In [ ]:
train_df.head()

In [ ]:
train_df.describe()

Con estas tablas podemos comprobar la distribución de los datos. Observamos que hay varios valores faltantes para los *min* de las variables. Esto se refiere a la existencia de valores *missing* que tendremos que tratar antes de seguir con los modelos.

In [ ]:
train_df.info()

El *set* de *train* contiene 59400 observaciones y 41 columnas. 
La columna *status_group* muestra la etiqueta para cada bomba, las otras 40 variables corresponden a las características, de las cuales 10 son numéricas y el resto categóricas. 
Comenzaremos investigando las numéricas.

## 4. Primera estimación de la exactitud o *accuracy*

A continuación observaremos la distribución de la variable objetivo *label* en *train* que nos servirá para los cálculos de nuestras predicciones.

In [ ]:
label_dict = {"functional":2,"functional needs repair":1,"non functional":0}
train_df["label"] = train_df["status_group"].map(label_dict)
sns.distplot(train_df["label"],kde=False)

In [ ]:
majority_class = train_df['status_group'].mode()[0]
print("The most frequent label is", majority_class)

y_prelim_pred = np.full(shape=train_df['status_group'].shape, fill_value=majority_class)
accuracy_score(train_df['status_group'], y_prelim_pred)

Significa que podemos empezar a hacer una estimación del 54.31% de probabilidad de que una bomba cualquiera de esta base de datos funcione bien (es decir, sea *functional*). Esto nos sirve de base para futuras predicciones.

Dado que nuestra variable objetivo es discreta, necesitaremos un algoritmo de clasificación supervisada, que son los que aplicaremos más adelante.

## 5. Variables numéricas

In [ ]:
numerical_vars = [col for col in train_df.columns if 
                train_df[col].dtype in ['int64', 'float64']]

### 5.1 *construction_year*

Vamos a realizar un gráfico de esta variable y el número de bombas construidas en ese año.

In [ ]:
sns.countplot(x=train_df["construction_year"],hue=train_df["status_group"])
plt.xticks(rotation=45, 
    horizontalalignment='right')
plt.title("Number of pumps constructed over the years", fontsize=14)
plt.xlabel("Construction year", fontsize=12)
plt.ylabel("Number of pumps constructed", fontsize=12)

Observamos que la mayoría de las que fueron construidas en 1985 no funcionan, mientras que las más recientes sí. Esto significa que este atributo puede ser muy útil a la hora de realizar las predicciones en nuestro modelo. El número de bombas que necesitan una reparación no parece muy elevado y sobretodo se mantiene estable con el paso de los años. Las filas que tienen un cero como año de construcción deberían revisarse.

### 5.2 *amount_tsh*

Esta variable muestra la cantidad de agua que queda en un pozo. Podría ser útil a la hora de predecir si su respectiva bomba funciona o no.

In [ ]:
sns.scatterplot(y=train_df["amount_tsh"],x=train_df["status_group"])

Si *amount_tsh* es mayor de 150000, lo más probable es que la bomba funcione.

### 5.3 Distribución de variables numéricas

In [ ]:
fig = plt.figure(figsize=(12,18))
sns.distributions._has_statsmodels=False
for i in range(len(numerical_vars)):
    fig.add_subplot(9,4,i+1)
    sns.distplot(train_df[numerical_vars].iloc[:,i].dropna())
    plt.xlabel(numerical_vars[i])

plt.tight_layout()
plt.show()

### 5.4 Valores atípicos

La visualización de los datos hace sospechar la posible existencia de valores atípicos o *outliers* en nuestros datos:

#### 5.4.1 Análisis univariante: diagramas de caja para atributos numéricos

In [ ]:
fig = plt.figure(figsize=(12, 18))

for i in range(len(numerical_vars)):
    fig.add_subplot(9, 4, i+1)
    sns.boxplot(y=train_df[numerical_vars].iloc[:,i])

plt.tight_layout()
plt.show()

Encontramos, por ejemplo, el siguiente: 
- *population* > 200000.

#### 5.4.2 Análisis de datos bivariantes: diagramas de dispersión para el objetivo frente a los atributos numéricos

In [ ]:
f = plt.figure(figsize=(14,20))

for i in range(len(numerical_vars)):
    f.add_subplot(9, 4, i+1)
    sns.scatterplot(train_df[numerical_vars].iloc[:,i], train_df["label"])
    
plt.tight_layout()
plt.show()

Observamos los siguientes:
- *amount_tsh* (> 200000) 
- *population* (> 13000)

### 5.5 Correlación entre variables

La correlación entre variables se muestra en el presente apartado.
Cabe destacar que esta puede verse afectada por la presencia de *outliers*.

Para poder utilizar la regresión lineal es necesario quitar las variables que estén altamente correlacionadas para así mejorar el modelo.

A continuación se presenta el gráfico de correlación:

In [ ]:
correlation = train_df.corr()

f, ax = plt.subplots(figsize=(8,6))
plt.title('Correlation of numerical attributes', size=12)
sns.heatmap(correlation)

Observamos que la correlación entre *district_code* y *region_code* es bastante elevada. Por lo que es posible que se deba eliminar una de las dos.

La correlación existente entre *construction_year* y *gps_height* también es elevada, pero estas dos variables no tienen una relación tan evidente, por lo que lo investigaremos más en profundidad antes de tomar cualquier decisión.

En relación a *label*, las variables más correlacionadas son:

In [ ]:
correlation['label'].sort_values(ascending=False)

La correlación lineal negativa de *region_code* con la variable objetivo es más alta que la de *district_code*. Mantenemos la que la tiene más elevada.

La correlación lineal con la variable objetivo es bastante baja en todas las variables pero puede significar que exista una correlación no lineal.

### 5.6 Valores faltantes en las variables numéricas

In [ ]:
train_df[numerical_vars].isna().sum().sort_values(ascending=False)

##### Population

In [ ]:
len(train_df.population[train_df.population == 0])

Una posible solución al problema de los valores faltantes podría ser transformarla en categórica.

## 6. Variables categóricas

In [ ]:
cat_vars = train_df.select_dtypes(include='object').columns
print(cat_vars)

### 6.1 Valores faltantes en las variables categóricas

In [ ]:
train_df[cat_vars].isna().sum().sort_values(ascending=False)

Visualizamos las categorías de *scheme_management*:

In [ ]:
sns.countplot(x='scheme_management', data=train_df)
plt.xticks(rotation=90)
plt.ylabel('Frequency')
plt.show()

## 7. Exportamos los datos tras el análisis

In [ ]:
train_df.to_csv("train_df_after_EDA.csv", index=False)
X_test.to_csv("X_test_after_EDA.csv", index=False)

# II. Preprocesado de datos
En el presente apartado vamos a ocuparnos del preprocesado de datos, es decir, prepararlos y optimizarlos para hacer los futuros *tests* con los modelos de *Machine Learning*.
## 1. Importación de liberías y datos

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
%matplotlib inline
print("Setup Complete")

A continuación cargamos los datos después del análisis:

In [ ]:
train_df = pd.read_csv("train_df_after_EDA.csv")
X_test = pd.read_csv("X_test_after_EDA.csv")

## 2. Extracción de variables similares

Los siguientes atributos

- *(extraction_type, extraction_type_group, extraction_type_class),*
- *(payment, payment_type),*
- *(water_quality, quality_group),*
- *(source, source_class),*
- *(subvillage, region, region_code, district_code, lga, ward),*
- *(waterpoint_type, waterpoint_type_group)*
- *(scheme_name, scheme_management)*

aportan una información muy similar, que indica que entre ellos hay una correlación elevada. Dejándolos, arriesgaríamos un sobreajuste.

Además:

- *num_private* consiste en 99% de ceros y no tiene una descripción clara, así que no podemos interpretarlo
- *wpt_name* no es muy informativa ya que tiene menos valores que el número de observaciones

Quitaremos una variable entre *district_code* y *region_code* por existir una elevada correlación entre ellas. Elegiremos la que tenga una correlación más elevada con la variable objetivo. La correlación negativa con la objetivo de *region_code* es más alta que la de *district_code*.

In [ ]:
train_df = train_df.drop(['installer','management_group','status_group','id_x','id_y', 'num_private', 'wpt_name', 
          'recorded_by', 'subvillage', 'scheme_name', 'region', 
          'quantity', 'water_quality', 'lga','ward', 'source_type', 'payment', 
          'waterpoint_type_group','extraction_type_group','extraction_type_class'],axis=1)
X_test = X_test.drop(['installer','management_group','id', 'num_private', 'wpt_name', 
          'recorded_by', 'subvillage', 'scheme_name', 'region', 
          'quantity', 'water_quality', 'lga','ward', 'source_type', 'payment', 
          'waterpoint_type_group','extraction_type_group','extraction_type_class'],axis=1)

In [ ]:
train_df.head()

## 3. Valores faltantes

In [ ]:
train_df["scheme_management"].fillna("unknown", inplace = True)
train_df["public_meeting"].fillna("unknown", inplace = True)
train_df["permit"].fillna("unknown", inplace = True)
train_df["funder"].fillna("unknown", inplace = True)

X_test["scheme_management"].fillna("unknown", inplace = True)
X_test["public_meeting"].fillna("unknown", inplace = True)
X_test["permit"].fillna("unknown", inplace = True)
X_test["funder"].fillna("unknown", inplace = True)

In [ ]:
X_test.isna().sum().sort_values(ascending=False)

## 4. Codificación ordinal de datos categóricos
Se decide utilizar esta metodología para evitar crear demasiadas columnas y dar algo de lógica al modelo a la hora de evaluar las características. Por ejemplo, en la variable *quality_group*, cuanto mayor sea la categoría, mejor la calidad del agua y más probable que la bomba funcione de manera correcta.

### 4.1 *quality_group*

In [ ]:
train_df.quality_group.value_counts()

In [ ]:
order_dict_quality = {"good":3,"salty":2,"milky":2,"colored":2,"fluoride":2,"unknown":1}
train_df["quality_group_code"] = [order_dict_quality[item] for item in train_df.quality_group]
del train_df["quality_group"]

X_test["quality_group_code"] = [order_dict_quality[item] for item in X_test.quality_group]
del X_test["quality_group"]

### 4.2 *quantity_group*

In [ ]:
train_df.quantity_group.value_counts()

In [ ]:
order_dict_quantity = {"enough":3,"insufficient":2,"dry":2,"seasonal":2,"unknown":1}
train_df["quantity_group_code"] = [order_dict_quantity[item] for item in train_df.quantity_group] 
del train_df["quantity_group"]

X_test["quantity_group_code"] = [order_dict_quantity[item] for item in X_test.quantity_group] 
del X_test["quantity_group"]

### 4.3 *payment_type*

In [ ]:
train_df.payment_type.value_counts()

In [ ]:
order_dict_payment = {"monthly":4,"annually":4,"on failure":3,"per bucket":3,"never pay":2,"unknown":1,"other":1}
train_df["payment_code"] = [order_dict_payment[item] for item in train_df.payment_type] 
del train_df["payment_type"]

X_test["payment_code"] = [order_dict_payment[item] for item in X_test.payment_type] 
del X_test["payment_type"]

### 4.4 *public_meeting*

In [ ]:
train_df.public_meeting.value_counts()

In [ ]:
order_dict_pub_meet = {True:2,False:1,"unknown":0}
train_df["public_meeting_code"] = [order_dict_pub_meet[item] for item in train_df.public_meeting] 
del train_df["public_meeting"]

X_test["public_meeting_code"] = [order_dict_pub_meet[item] for item in X_test.public_meeting] 
del X_test["public_meeting"]

### 4.5 *permit*

In [ ]:
train_df.permit.value_counts()

In [ ]:
order_dict_permit = {True:2,False:1,"unknown":0}
train_df["permit_code"] = [order_dict_pub_meet[item] for item in train_df.permit] 
del train_df["permit"]

X_test["permit_code"] = [order_dict_pub_meet[item] for item in X_test.permit] 
del X_test["permit"]

## 5. Más mejoras al modelo

A continuación crearemos variables nuevas (basadas en las características de nuestro *dataset*) que describirán de una forma más óptima a la objetivo.

### 5.1 *amount_tsh*
Después de nuestro Análisis Exploratorio de los datos hemos definido una condición para separar las bombas que funcionan de las que no. Vamos a proceder a crear una nueva variable binaria que refleje esta información.

In [ ]:
train_df.loc[train_df['amount_tsh'] < 200000, 'amount_tsh'] = 0
train_df.loc[train_df['amount_tsh'] >= 200000, 'amount_tsh'] = 1

X_test.loc[train_df['amount_tsh'] < 200000, 'amount_tsh'] = 0
X_test.loc[train_df['amount_tsh'] >= 200000, 'amount_tsh'] = 1

### 5.2 *construction_year*
A continuación transformamos *construction_year* en una variable categórica que contenga las siguientes décadas de años: '60s', '70s', '80s', '90s, '00s', '10s' y 'unknown' para los años desconocidos.

In [ ]:
def construction_wrangler(row):
    if row['construction_year'] >= 1960 and row['construction_year'] < 1970:
        return '60s'
    elif row['construction_year'] >= 1970 and row['construction_year'] < 1980:
        return '70s'
    elif row['construction_year'] >= 1980 and row['construction_year'] < 1990:
        return '80s'
    elif row['construction_year'] >= 1990 and row['construction_year'] < 2000:
        return '90s'
    elif row['construction_year'] >= 2000 and row['construction_year'] < 2010:
        return '00s'
    elif row['construction_year'] >= 2010:
        return '10s'
    else:
        return 'unknown'
    
train_df['construction_year'] = train_df.apply(lambda row: construction_wrangler(row), axis=1)
X_test['construction_year'] = X_test.apply(lambda row: construction_wrangler(row), axis=1)

### 5.3 *date_recorded*
Calcularemos el número de días que indica la variable *date_recorded* en el cual se obtuvieron los datos para una bomba en concreto, hasta la fecha más reciente del *dataset*. La idea es que es más probable que los guardados en nuestros datos en una época más reciente funcionen correctamente.
Iniciaremos convirtiendo la columna en una de tipo *datetime*.

In [ ]:
train_df.date_recorded = pd.to_datetime(train_df.date_recorded)
X_test.date_recorded = pd.to_datetime(X_test.date_recorded)

train_df.date_recorded.describe()

Los datos más recientes son del 12 de marzo de 2013.

In [ ]:
train_df['days_since_recorded'] = pd.datetime(2013, 12, 3) - pd.to_datetime(train_df.date_recorded)
train_df['days_since_recorded'] = train_df['days_since_recorded'].astype('timedelta64[D]').astype(int)

X_test['days_since_recorded'] = pd.datetime(2013, 12, 3) - pd.to_datetime(X_test.date_recorded)
X_test['days_since_recorded'] = X_test['days_since_recorded'].astype('timedelta64[D]').astype(int)

In [ ]:
train_df['days_since_recorded']

In [ ]:
train_df = train_df.drop("date_recorded",axis=1)
X_test = X_test.drop("date_recorded",axis=1)

In [ ]:
train_df.shape

In [ ]:
X_test.shape

## 6. Codificación de tipo *One-Hot* para las variables categóricas

Para variables categóricas donde no existe una relación de orden, la codificación mediante enteros no suele ser adecuada. En estos casos, se puede aplicar una codificación especial donde se agrega una nueva variable binaria (con valores verdadero o falso) para cada valor de categoría posible.
La codificación *One-Hot* es un método para etiquetar a qué clase pertenecen los datos y la idea es asignar 0 a toda la dimensión, excepto 1 para la clase a la que pertenecen los datos.

In [ ]:
cat_vars = train_df.select_dtypes(include='object').columns
print(cat_vars)
len(cat_vars)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Aplicamos el método One-Hot-Encoder para cada columna con datos de tipo categórico
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(train_df[cat_vars])).astype(np.int64)
OH_cols_test = pd.DataFrame(OH_encoder.transform(X_test[cat_vars])).astype(np.int64)

# El método quita el index, se vuelve a poner
OH_cols_train.index = train_df.index
OH_cols_test.index = X_test.index

OH_cols_train.columns = OH_encoder.get_feature_names(cat_vars)
OH_cols_test.columns = OH_encoder.get_feature_names(cat_vars)

# Se eliminan las columnas categóricas
num_X_train = train_df.drop(cat_vars, axis=1)
num_X_valid = X_test.drop(cat_vars, axis=1)

# Se añaden las columnas de One-Hot-Encoder a atributos numéricos
OH_train_df = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_test = pd.concat([num_X_valid, OH_cols_test], axis=1)

In [ ]:
OH_train_df.head()

## 7. Selección de atributos
### 7.1 Regularización con Regresión Logística

En la regularización Lasso, también llamada L1, la complejidad C se mide como la media del valor absoluto de los coeficientes del modelo. Con ella favorecemos que algunos de los coeficientes acaben valiendo 0. Esto puede ser útil para descubrir cuáles de los atributos de entrada son relevantes y, en general, para obtener un modelo que generalice mejor. Lasso nos ayuda a hacer la selección de atributos de entrada.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel

X, y = OH_train_df[OH_train_df.columns.drop("label")], OH_train_df['label']

logistic = LogisticRegression(solver="saga",C=1, penalty="l1", random_state=7).fit(X, y)
model = SelectFromModel(logistic, prefit=True)

X_new = model.transform(X)
X_new

In [ ]:
selected_features = pd.DataFrame(model.inverse_transform(X_new), 
                                 index=X.index,
                                 columns=X.columns)

selected_columns = selected_features.columns[selected_features.var() != 0]

In [ ]:
len(selected_columns)

In [ ]:
selected_columns

In [ ]:
train_df_selected_features = OH_train_df[selected_columns].join(y)

In [ ]:
X_test_selected_features = OH_X_test[selected_columns]

### 7.2 Importancia de atributos con *Random Forest*

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(criterion='gini',min_samples_split=8, n_estimators=1000,
                           random_state = 7)
rf.fit(X, y)

In [ ]:
def imp_df(column_names, importances):
    df = pd.DataFrame({'feature': column_names,
                       'feature_importance': importances}) \
           .sort_values('feature_importance', ascending = False) \
           .reset_index(drop = True)
    return df

def var_imp_plot(imp_df, title):
    imp_df.columns = ['feature', 'feature_importance']
    sns.barplot(x = 'feature_importance', y = 'feature', data = imp_df, orient = 'h', color = 'royalblue') \
       .set_title(title, fontsize = 20)

In [ ]:
base_imp = imp_df(X.columns, rf.feature_importances_)
top_30_imp = base_imp[0:30]
top_30_features = top_30_imp.feature

In [ ]:
pylab.rcParams["figure.figsize"] = (10,10)

In [ ]:
var_imp_plot(base_imp, 'Default feature importance (scikit-learn)')

In [ ]:
train_df_final_top_imp = OH_train_df[top_30_features].join(y)
X_test_final_top_imp = OH_X_test[top_30_features]

In [ ]:
train_df_final_top_imp.shape

In [ ]:
train_df_selected_features.shape

## 8. Exportación del *dataframe* final

In [ ]:
train_df_selected_features.to_csv("train_df_final.csv", index=False)
X_test_selected_features.to_csv("X_test_final.csv", index=False)

En la parte tres y última de este trabajo se procede a seleccionar el modelo.
Se han hecho pruebas con diferentes modelos de 30 y 80 variables y los resultados parecen ser estables en todos los modelos. El *set* de 80 variables que obtuvimos después de hacer la regularización con regresión logística cada vez tiene una puntuación más alta, por lo que se ha decidido utilizar los *dataframes* de *selected_features* como el *input* final.

# III. Selección del modelo y entrega
## 1. Preparación de los datos
### 1.1 Importación de librerías y datos

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler as ss
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV


pd.set_option('display.max_columns', None)

# Machine Learning

# Árboles de decisión    
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier

# Métodos de Ensamble
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost
from xgboost import XGBClassifier

# Procesos Gaussianos
from sklearn.gaussian_process import GaussianProcessClassifier
    
# Generalized Linear Model (GLM)
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import RidgeClassifierCV
from sklearn.linear_model import Perceptron   
    
# K-Nearest Neighbors
from sklearn.neighbors import KNeighborsClassifier
    
# Máquinas de Vectores de Soporte (SVM)
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.svm import NuSVC

# Análisis Discriminante
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

# Clasificador Naive Bayes
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB

# Métrica
from sklearn.metrics import accuracy_score, confusion_matrix

# Análisis de Componentes Principales
from sklearn import decomposition

print("Setup Complete")

In [ ]:
train_df_final = pd.read_csv("train_df_final.csv")
X_test_final = pd.read_csv("X_test_final.csv")

In [ ]:
X_test_final.shape

In [ ]:
train_df_final.shape

### 1.2 División de datos en *train* y *test*

In [ ]:
X = train_df_final.drop("label",axis=1)
y = train_df_final["label"]

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2, stratify=y, random_state=42)

In [ ]:
X.isnull().values.any()

### 1.3 *Standard Scaling*

Es un método que transforma tus datos de tal manera que su distribución tenga una media de 0 y desviación estándar de 1.
En el caso de datos multivariantes, esto se realiza de manera independiente columna por columna.
Información obtenida de:
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

In [ ]:
sc = ss()
X_train = sc.fit_transform(X_train)
X_valid = sc.transform(X_valid)
X_test = sc.transform(X_test_final)

### 1.4 Análisis de Componentes Principales (o PCA en inglés)
Creemos que es importante mencionarlo a pesar de que no mejoró la puntuación final, por lo que no fue incluido en el modelo.

In [ ]:
pca = decomposition.PCA(.95)

In [ ]:
pca.fit(X_train)

In [ ]:
pca.n_components_

In [ ]:
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)
X_valid_pca = pca.transform(X_valid)

## 2. Selección del modelo
Comprobamos los diferentes modelos en el *validation set*:

### 2.1 Árboles de decisión

In [ ]:
# 1

decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, y_train)
y_pred = decision_tree.predict(X_valid)

acc_decision_tree = round(accuracy_score(y_valid,y_pred) * 100, 2)
acc_decision_tree

In [ ]:
# 2

extra_tree = DecisionTreeClassifier()
extra_tree.fit(X_train, y_train)
y_pred = extra_tree.predict(X_valid)

acc_extra_tree = round(accuracy_score(y_valid,y_pred) * 100, 2)
acc_extra_tree

### 2.2 Ensamblados

In [ ]:
# RandomForest

rfc = RandomForestClassifier(criterion='entropy', n_estimators = 1000,min_samples_split=8,random_state=42,verbose=5)
rfc.fit(X_train, y_train)

y_pred = rfc.predict(X_valid)

acc_rfc = round(accuracy_score(y_valid,y_pred) * 100, 2)
acc_rfc

Resultado con PCA y 30 variables: 77.49

Resultado sin PCA y 30 variables: 79.21

Resultado con PCA y 80 variables: 77.53

Resultado sin PCA y 80 variables: 79.71

In [ ]:
# GradientBoostingClassifier

GB = GradientBoostingClassifier(n_estimators=100, learning_rate=0.075, 
                                max_depth=13,max_features=0.5,
                                min_samples_leaf=14, verbose=5)

GB.fit(X_train, y_train)     
y_pred = GB.predict(X_valid)

acc_GB = round(accuracy_score(y_valid,y_pred) * 100, 2)
acc_GB

Resultado con PCA y 30 variables: 77.21

Resultado sin PCA y 30 variables: 78.95

Resultado sin PCA y 80 variables: 79.19

In [ ]:
# LightGBM

LGB = LGBMClassifier(objective='multiclass', learning_rate=0.75, num_iterations=100, 
                     num_leaves=50, random_state=123, max_depth=8)

LGB.fit(X_train, y_train)
y_pred = LGB.predict(X_valid)

acc_LGB = round(accuracy_score(y_valid,y_pred) * 100, 2)
acc_LGB

Resultado con PCA y with 30 variables: 76.52

Resultado sin PCA y 30 variables: 77.88

Resultado sin PCA y 80 variables: 78.47

In [ ]:
# AdaBoostClassifier

AB = AdaBoostClassifier(n_estimators=100, learning_rate=0.075)
AB.fit(X_train, y_train)     
y_pred = AB.predict(X_valid)

acc_AB = round(accuracy_score(y_valid,y_pred) * 100, 2)
acc_AB

In [ ]:
# BaggingClassifier

BC = BaggingClassifier(n_estimators=100)
BC.fit(X_train_pca, y_train)     
y_pred = BC.predict(X_valid_pca)

acc_BC = round(accuracy_score(y_valid,y_pred) * 100, 2)
acc_BC

Resultado con PCA: 75.98

Resultado sin PCA: 77.33

In [ ]:
# XGBoost

xgb = XGBClassifier(n_estimators=1000, learning_rate=0.05, n_jobs=5)
xgb.fit(X_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(X_valid, y_valid)], 
             verbose=False)

y_pred = xgb.predict(X_valid)
acc_xgb = round(accuracy_score(y_valid,y_pred) * 100, 2)
acc_xgb

Resultado con PCA y 30 variables: 74.3
    
Resultado sin PCA y 30 variables: 75.45
    
Resultado sin PCA y 80 variables: 76.1

In [ ]:
# ExtraTreesClassifier

ETC = ExtraTreesClassifier(n_estimators=100)
ETC.fit(X_train, y_train)     
y_pred = ETC.predict(X_valid)

acc_ETC = round(accuracy_score(y_valid,y_pred) * 100, 2)
acc_ETC

### 2.3 *Generalized Linear Models* (GLM)

In [ ]:
# LogisticRegression

LG = LogisticRegression(solver="lbfgs", multi_class="multinomial")
LG.fit(X_train, y_train)     
y_pred = LG.predict(X_valid)

acc_LG = round(accuracy_score(y_valid,y_pred) * 100, 2)
acc_LG

Podemos utilizar la Regresión Logística para validar nuestras decisiones de creación de atributos. Esto se puede hacer calculando el coeficiente de los atributos en la función de decisión.
Los coeficientes positivos mejoran la respuesta (aumentan la probabilidad) y los negativos empeoran la respuesta (disminuyen la probabilidad).

In [ ]:
coeff_df = pd.DataFrame(train_df_final.columns.delete(0))
coeff_df.columns = ['Feature']
coeff_df["Correlation"] = pd.Series(LG.coef_[0])

coeff_df.sort_values(by='Correlation', ascending=False)

In [ ]:
# PassiveAggressiveClassifier

PAC = PassiveAggressiveClassifier()
PAC.fit(X_train, y_train)
y_pred = PAC.predict(X_valid)

acc_PAC = round(accuracy_score(y_valid,y_pred) * 100, 2)
acc_PAC

In [ ]:
# RidgeClassifierCV

RC = RidgeClassifierCV()
RC.fit(X_train, y_train)
y_pred = RC.predict(X_valid)

acc_RC = round(accuracy_score(y_valid,y_pred) * 100, 2)
acc_RC

In [ ]:
# Perceptron

P = Perceptron()
P.fit(X_train, y_train)
y_pred = P.predict(X_valid)

acc_P = round(accuracy_score(y_valid,y_pred) * 100, 2)
acc_P

In [ ]:
# StochasticGradientDescent

SGD = SGDClassifier(shuffle=True,average=True)
SGD.fit(X_train, y_train)
y_pred = SGD.predict(X_valid)

acc_SGD = round(accuracy_score(y_valid,y_pred) * 100, 2)
acc_SGD

### 2.4 KNN

In [ ]:
# KNN

knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_valid)

acc_knn = round(accuracy_score(y_valid,y_pred) * 100, 2)
acc_knn

### 2.5 Análisis Discriminante

In [ ]:
# LinearDiscriminantAnalysis

LDA = LinearDiscriminantAnalysis()
LDA.fit(X_train,y_train)
LDA.predict(X_valid)

acc_LDA = round(accuracy_score(y_valid,y_pred) * 100, 2)
acc_LDA

In [ ]:
# QuadraticDiscriminantAnalysis

QDA = QuadraticDiscriminantAnalysis()
QDA.fit(X_train,y_train)
QDA.predict(X_valid)

acc_QDA = round(accuracy_score(y_valid,y_pred) * 100, 2)
acc_QDA

### 2.6 Clasificador *Naive Bayes*

In [ ]:
# BernoulliNB

bernoulliNB = BernoulliNB()
bernoulliNB.fit(X_train,y_train)
bernoulliNB.predict(X_valid)

acc_bernoulliNB = round(accuracy_score(y_valid,y_pred) * 100, 2)
acc_bernoulliNB

In [ ]:
# GaussianNB

gaussianNB = GaussianNB()
gaussianNB.fit(X_train,y_train)
gaussianNB.predict(X_valid)

acc_gaussianNB = round(accuracy_score(y_valid,y_pred) * 100, 2)
acc_gaussianNB

## 3. Comparación de los resultados de los modelos

A continuación representaremos gráficamente todos los modelos:

In [ ]:
models = pd.DataFrame({
    'Model': ['LightGBM','Decision Tree',"Extra Tree",'Random Forest', 'KNN', 'Logistic Regression', 
              'Stochastic Gradient Decent',"XGBoost", "Ada Boost Classifier", 
              "Bagging Classifier", "Passive Agressive Cl", "Ridge","Perceptron",
              'Gradient Boosting Classifier','Extra Trees',
              "LinearDA","QuadraticDA","BernoulliNB","GaussianNB"],
    'Score': [acc_LGB,acc_decision_tree,acc_extra_tree,acc_rfc, acc_knn, acc_LG,
              acc_SGD, acc_xgb, acc_AB, 
              acc_BC, acc_PAC, acc_RC, acc_P,
              acc_GB, acc_ETC,
             acc_LDA, acc_QDA, acc_bernoulliNB, acc_gaussianNB]})
sorted_by_score = models.sort_values(by='Score', ascending=False)

In [ ]:
sns.barplot(x='Score', y = 'Model', data = sorted_by_score, color = 'g')
plt.title('Exactitud del algoritmo \n')
plt.xlabel('Exactitud en test (%)')
plt.ylabel('Modelo')

Como podemos comprobar, los tres mejores modelos son:

**- *Gradient Boosting Classifier* - 80.01**

**- *Random Forest* - 80.0**

**- *Light GBM* - 78.7**

De ellos, el *Gradient Boosting Classifier* es el mejor y más rápido ya que *Random Forest* da una puntuación un poco peor (de 80.0 comparado a 80.01 del *Gradient Boosting Classifier*).

## 4. Tuneado de modelos y ajuste de parámetros
Se ha decidido ajustar los parámetros basándonos en el tuneado de los tres modelos que acabamos de mencionar.

In [ ]:
sc = ss()
X = sc.fit_transform(X)
X_test = sc.transform(X_test_final)

# RandomForest

rfc = RandomForestClassifier(criterion='entropy',min_samples_split=8, n_estimators=1000)

rfc.fit(X, y)     

In [ ]:
# GradientBoostingClassifier

GB = GradientBoostingClassifier(n_estimators=150, learning_rate=0.05, max_depth=14,max_features=0.5,min_samples_leaf=14,verbose=5)

GB.fit(X, y)     

In [ ]:
# Lightgbm

LGB = LGBMClassifier(objective='multiclass', learning_rate=0.75, num_iterations=100, 
                     num_leaves=40, random_state=123,max_depth=15)

LGB.fit(X, y)

## 5. Envío
Para finalizar, procedemos a crear el archivo que subiremos a la *web* de *DrivenData* con el formato que se nos indica en *Submission_format.csv*:

In [ ]:
submission_df = pd.read_csv("Submission_format.csv")

In [ ]:
X_test = sc.transform(X_test_final)
submission_df['status_group']=rfc.predict(X_test)

In [ ]:
vals_to_replace = {2:'functional', 1:'functional needs repair', 0:'non functional'}

submission_df.status_group = submission_df.status_group.replace(vals_to_replace)

In [ ]:
submission_df.to_csv("submission_Gemma.csv",sep=',', index=False)

## 6. Conclusiones

El objetivo de este proyecto era predecir si una bomba funcionaba o no o si requería de una reparación basándonos en datos que describían la bomba, el pozo, sus alrededores, quién la gestionaba y la fecha.

Hemos iniciado con un Análisis Exploratorio de los Datos. Calculando la exactitud y dividiendo los datos en numéricos y categóricos según su tipología. A continuación hemos identificado los valores faltantes para lidiar con ellos en la fase posterior del preprocesado, buscado valores atípicos y analizado y actuado sobre las diferentes correlaciones existentes entre algunos de los atributos.

En el siguiente paso hemos realizado la limpieza y preprocesado de los datos. Hemos iniciado eliminando atributos que contenían información similar para evitar multicolinealidad. Después hemos tratado los datos faltantes, realizado codificación de tipo ordinal para las que lo requerían y de tipo *One-Hot* para las demás. Finalmente, hemos creado nuevas variables que definieran mejor a la objetivo.

Acabado el preprocesado, hemos seleccionado con Regresión Logística las 80 variables más importantes de un total de 90 columnas. Para finalizar, se han comprobado diversos modelos y mostrado los resultados en un gráfico, obteniendo que los mejores son: 
 
**- Gradient Boosting Classifier - 80.01**

**- Random Forest - 80.0**

**- Light GBM - 78.7**